In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import dabl 
import matplotlib.pyplot as plt
import dc_stat_think as dcst

from sklearn.metrics import confusion_matrix, classification_report

Part A
1. Train the model with the different regularizer that was taught in class -
L1, L2, Dropout, and Early stopping
a. Make a table and show the difference in metrics
2. Train the models with different optimizers - SGD, RMSProp and ADAM
a. Tweak the learning rate and momentum
b. Note down the difference in metrics for each
3. Make a table of Optimal Hyperparamets  [ no of epochs, best
regulariser, the best optimizer ]

In [2]:
df = pd.read_csv('../data/diabetes.csv')

In [3]:
Y = df.Outcome

X = df.drop('Outcome',axis=1)
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [4]:

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=45)

In [5]:
def create_baseline():
    
    model = Sequential()
    model.add(Dense(10,input_dim=8,activation='relu'))
    model.add(Dense(10,input_dim=10,activation='relu'))
    model.add(Dense(10,input_dim=10,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [6]:
estimator = KerasClassifier(build_fn=create_baseline,epochs=20,batch_size=5,verbose=0)

kfold = StratifiedKFold(n_splits=5,shuffle=True)
results = cross_val_score(estimator,X_scaled,Y,cv=kfold)

print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

<ipython-input-6-58e2ee4cdf8c>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=create_baseline,epochs=20,batch_size=5,verbose=0)


Baseline: 75.77% (3.18%)


## 1
- Train the model with the different regularizer that was taught in class - L1, L2, Dropout, and Early stopping a. Make a table and show the difference in metrics

In [7]:
scores  = {}


In [8]:
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [9]:
### baseline with no regularizers added 

In [10]:
def create_baseline():
    
    model = Sequential()
    model.add(Dense(10,input_dim=8,activation='relu'))
    model.add(Dense(10,input_dim=10,activation='relu'))
    model.add(Dense(10,input_dim=10,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model
estimator = KerasClassifier(build_fn=create_baseline,epochs=50,batch_size=5,verbose=0)

kfold = StratifiedKFold(n_splits=5,shuffle=True)
results = cross_val_score(estimator,X_scaled,Y,cv=kfold)

print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

scores['baseline']= [results.mean()*100, results.std()*100]

<ipython-input-10-2e87a21885d4>:11: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=create_baseline,epochs=50,batch_size=5,verbose=0)


Baseline: 74.61% (2.31%)


In [11]:
scores

{'baseline': [74.60826873779297, 2.3072956225034953]}

### 1.1 L1 loss added to all layers

In [12]:
def create_baseline():
    
    model = Sequential()
    model.add(Dense(10,input_dim=8,activation='relu',kernel_regularizer=regularizers.L1(0.01)))
    model.add(Dense(10,input_dim=10,activation='relu',kernel_regularizer=regularizers.L1(0.01)))
    model.add(Dense(10,input_dim=10,activation='relu',kernel_regularizer=regularizers.L1(0.01)))
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

estimator = KerasClassifier(build_fn=create_baseline,epochs=50,batch_size=5,verbose=0)

kfold = StratifiedKFold(n_splits=5,shuffle=True)
results = cross_val_score(estimator,X_scaled,Y,cv=kfold)

print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
scores['L1'] = [results.mean()*100, results.std()*100]

<ipython-input-12-d29df8d5357e>:12: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=create_baseline,epochs=50,batch_size=5,verbose=0)


Baseline: 77.74% (2.76%)


In [13]:

scores

{'baseline': [74.60826873779297, 2.3072956225034953],
 'L1': [77.73533701896667, 2.759601550404463]}

### l1 regulaization where [ loss = l1 * reduce_sum(abs(x)) ] increased the baseline score by 2 % and decreased the standard deviation by 2 % 


### 1.2 L2 regularizer 

In [14]:
def create_baseline():
    
    model = Sequential()
    model.add(Dense(10,input_dim=8,activation='relu',kernel_regularizer=regularizers.L2(0.01)))
    model.add(Dense(10,input_dim=10,activation='relu',kernel_regularizer=regularizers.L2(0.01)))
    model.add(Dense(10,input_dim=10,activation='relu',kernel_regularizer=regularizers.L2(0.01)))
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

estimator = KerasClassifier(build_fn=create_baseline,epochs=50,batch_size=5,verbose=0)

kfold = StratifiedKFold(n_splits=5,shuffle=True)
results = cross_val_score(estimator,X_scaled,Y,cv=kfold)

print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

scores['L2'] = [results.mean()*100, results.std()*100]

<ipython-input-14-c0fb14970cb4>:12: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=create_baseline,epochs=50,batch_size=5,verbose=0)


Baseline: 77.08% (1.73%)


### 1.3 Dropout 

#### 1.3.1 all layers given drop out .5

#### 1.3.2 dropout after last layer added

In [15]:
def create_baseline():
    
    model = Sequential()
    model.add(Dense(10,input_dim=8,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10,input_dim=10,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10,input_dim=10,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

estimator = KerasClassifier(build_fn=create_baseline,epochs=50,batch_size=5,verbose=0)

kfold = StratifiedKFold(n_splits=5,shuffle=True)
results = cross_val_score(estimator,X_scaled,Y,cv=kfold)

print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

scores['Dropout all layers'] = [results.mean()*100, results.std()*100]

<ipython-input-15-459099051837>:15: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=create_baseline,epochs=50,batch_size=5,verbose=0)


Baseline: 76.69% (1.73%)


#### 1.3.2 dropout after last layer added

In [16]:


def create_baseline():
    
    model = Sequential()
    model.add(Dense(10,input_dim=8,activation='relu'))

    model.add(Dense(10,input_dim=10,activation='relu'))

    model.add(Dense(10,input_dim=10,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

estimator = KerasClassifier(build_fn=create_baseline,epochs=50,batch_size=5,verbose=0)

kfold = StratifiedKFold(n_splits=5,shuffle=True)
results = cross_val_score(estimator,X_scaled,Y,cv=kfold)

print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

scores['Dropout_last_layer'] = [results.mean()*100, results.std()*100]

<ipython-input-16-95682f48c688>:15: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=create_baseline,epochs=50,batch_size=5,verbose=0)


Baseline: 76.16% (4.72%)


### 1.4 early stopping



In [17]:


def create_baseline():
    
    model = Sequential()
    model.add(Dense(10,input_dim=8,activation='relu',kernel_regularizer=regularizers.L2(0.01)))

    model.add(Dense(10,input_dim=10,activation='relu',kernel_regularizer=regularizers.L2(0.01)))

    model.add(Dense(10,input_dim=10,activation='relu',kernel_regularizer=regularizers.L2(0.01)))
 
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

# add early stoping to stop training if loss does not decrease in 3 epocs 
callback = EarlyStopping(monitor='loss', patience=3)
estimator = KerasClassifier(build_fn=create_baseline,epochs=50,batch_size=5,verbose=0,callbacks=[callback])

kfold = StratifiedKFold(n_splits=5,shuffle=True)
results = cross_val_score(estimator,X_scaled,Y,cv=kfold)

print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

scores['Early_stopping_layer'] = [results.mean()*100, results.std()*100]

<ipython-input-17-19f43ef1346e>:17: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=create_baseline,epochs=50,batch_size=5,verbose=0,callbacks=[callback])


Baseline: 76.17% (3.14%)


In [18]:


def create_baseline():
    
    model = Sequential()
    model.add(Dense(10,input_dim=8,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10,input_dim=10,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10,input_dim=10,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

# add early stoping to stop training if loss does not decrease in 3 epocs 
callback = EarlyStopping(monitor='loss', patience=3)
estimator = KerasClassifier(build_fn=create_baseline,epochs=50,batch_size=5,verbose=0,callbacks=[callback])

kfold = StratifiedKFold(n_splits=5,shuffle=True)
results = cross_val_score(estimator,X_scaled,Y,cv=kfold)

print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

scores['all regularization added'] = [results.mean()*100, results.std()*100]

<ipython-input-18-744c93e36e52>:17: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=create_baseline,epochs=50,batch_size=5,verbose=0,callbacks=[callback])


Baseline: 69.01% (4.15%)


In [19]:


def create_baseline():
    
    model = Sequential()
    model.add(Dense(10,input_dim=8,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10,input_dim=10,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10,input_dim=10,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

# add early stoping to stop training if loss does not decrease in 3 epocs 
callback = EarlyStopping(monitor='loss', patience=3)
estimator = KerasClassifier(build_fn=create_baseline,epochs=100,batch_size=5,verbose=0,callbacks=[callback])

kfold = StratifiedKFold(n_splits=5,shuffle=True)
results = cross_val_score(estimator,X_scaled,Y,cv=kfold)

print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

scores['all regularization 100 epoch'] = [results.mean()*100, results.std()*100]

<ipython-input-19-671de014f390>:17: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=create_baseline,epochs=100,batch_size=5,verbose=0,callbacks=[callback])


Baseline: 66.28% (1.74%)


In [22]:


def create_baseline():
    
    model = Sequential()
    model.add(Dense(10,input_dim=8,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10,input_dim=10,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10,input_dim=10,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

# add early stoping to stop training if loss does not decrease in 3 epocs 
callback = EarlyStopping(monitor='loss', patience=3)
estimator = KerasClassifier(build_fn=create_baseline,epochs=20,batch_size=5,verbose=0,callbacks=[callback])

kfold = StratifiedKFold(n_splits=5,shuffle=True)
results = cross_val_score(estimator,X_scaled,Y,cv=kfold)

print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

scores['all regularization 20 epoch'] = [results.mean()*100, results.std()*100]

<ipython-input-22-b1ff4a65f237>:17: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=create_baseline,epochs=20,batch_size=5,verbose=0,callbacks=[callback])


Baseline: 65.75% (0.71%)


In [23]:
table =pd.DataFrame(scores).T
table.columns=['Avg_score','Std of scores']
table.sort_values(by='Avg_score',ascending=False)

,Avg_score,Std of scores
L1,77.735337,2.759602
L2,77.081742,1.733154
Dropout all layers,76.691282,1.732569
Early_stopping_layer,76.172652,3.140303
Dropout_last_layer,76.163313,4.722879
baseline,74.608269,2.307296
all regularization added,69.011968,4.152847
all regularization 100 epoch,66.277057,1.744875
all regularization 20 epoch,65.754181,0.705270


### Insight

The increases in scores from the different regularization techinques showed marginal gains and increases in score could not be statistically significat. classification reports and f1 scores might show more difference between the techniques. Using all techniques togeather generated lower scores than than using a single technique.